# SiRF Reader - Locosys SiRF Binary

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/gps-wizard/

In [1]:
import unittest

from file_reader import FileReader

## Main Class

In [2]:
class SirfReader(FileReader):
    '''SBN + SBP files - Locosys SiRF Binary'''

    def __init__(self, filename):
        '''Basic init just records the filename'''

        super().__init__(filename)


    def load(self):
        '''Load file into memory'''

        with open(self.filename, "rb") as f:
            self.buffer = f.read()
            self.bufferPtr = 0

            self.readHeader()
            self.readData()


    def readHeader(self):
        '''Read Locosys header into memory'''

        # Main header content
        message = self.readMessage()

        # Parse ASCII data, ignoring message[0] which is the SiRF message ID - typically 0xfd but sometimes 0xdf
        splitFields = message[1:].decode('ascii').split(',')
        
        # Decode fields that are always present, regardless of the message ID being 0xfd or 0xdf
        self.header['username'] = splitFields[0]
        self.header['serial'] = int(splitFields[1])
        self.header['frequency'] = 5 if splitFields[2] == '2' else 1
        self.header['firmware'] = splitFields[3]

        # Note that message ID 0xdf has 2 additional fields (maybe 3) which are currently ignored
        # e.g. "GW60USER,168605168,2,V1.3A0926C,0.0h,00,"


    def readMessage(self):
        '''Read SiRF message from buffer'''

        # Check the start token is a0a2
        startToken = self.readUnsigned16BE()
        if startToken != 0xa0a2:
            raise ValueError('Unexpected start token {:x} - expected {:x}'.format(startToken, 0xa0a2))

        # Internal length is sometimes big endian, sometimes little endian!
        length = self.readUnsigned16BE()
        if length > 255:
            length = length >> 8
            
        # Main message content
        payload = self.readBytes(length)

        # Evaluate checksum
        checksum = self.readUnsigned16BE()
        if checksum != sum(payload):
            raise ValueError('Checksum difference - {}'.format(checksum - sum(payload)))

        # Check the end token is b0b3
        endToken = self.readUnsigned16BE()
        if endToken != 0xb0b3:
            raise ValueError('Unexpected end token {:x} - expected {:x}'.format(endToken, 0xb0b3))

        return payload


    def readUnsigned16BE(self):
        '''Read unsigned 16-bit integer from the buffer - big endian'''
        
        value = self.buffer[self.bufferPtr] << 8 | self.buffer[self.bufferPtr + 1]
        self.bufferPtr += 2
        
        return value
        

    def readUnsigned16LE(self):
        '''Read unsigned 16-bit integer from the buffer - little endian'''
        
        value = self.buffer[self.bufferPtr + 1] << 8 | self.buffer[self.bufferPtr]
        self.bufferPtr += 2
        
        return value
        

    def readBytes(self, numBytes):
        '''Read fixed number of bytes from the buffer'''
        
        value = self.buffer[self.bufferPtr : self.bufferPtr + numBytes]
        self.bufferPtr += numBytes
        
        return value

        
    def consumeRawData(self):
        '''Consume raw SiRF data - convert to standard data structure'''

        # Convert HDOP from integer to decimal
        self.data['hdop'] = self.rawData['hdop'] / 5

        # Satellite count is already an integer
        self.data['sat'] = self.rawData['sv_count']

        # Copy satellite IDs
        self.data['sv_ids'] = self.rawData['sv_ids']

        # Convert latitude and longitude to decimal
        self.data['lat'] = self.rawData['latitude'] / 10000000
        self.data['lon'] = self.rawData['longitude'] / 10000000

        # Convert elevation and SOG from cm/s to m/s
        self.data['ele'] = self.rawData['altitude_msl'] / 100
        self.data['sog'] = self.rawData['sog'] / 100

        # Convert COG from integer to decimal
        self.data['cog'] = self.rawData['cog'] / 100

        # Convert rate of climb from cm/s to m/s
        self.data['roc'] = self.rawData['climb_rate'] / 100

        # Convert SDOP from cm/s to m/s - original SBN format was without SDOP
        if 'sdop' in self.rawData.dtype.names:
            self.data['sdop'] = self.rawData['sdop'] / 100

        # Convert VSDOP from cm/s to m/s - original SBN format was without VSDOP
        if 'vsdop' in self.rawData.dtype.names:
            self.data['vsdop'] = self.rawData['vsdop'] / 100

## Unit Tests

In [3]:
class TestInit(unittest.TestCase):
    '''Class to test init'''

    def testFilename(self):
        '''Test the filename is as expected'''

        self.assertEqual(sirfReader.filename, 'test.gpx')


    def testHeader(self):
        '''Test the header is as expected'''

        self.assertEqual(sirfReader.header, {})


    def testData(self):
        '''Test the data is as expected'''

        self.assertEqual(sirfReader.data, {})


    def testNumRecords(self):
        '''Test the number of records is zero'''

        self.assertEqual(sirfReader.numRecords, 0)

In [4]:
if __name__ == '__main__':
    # Determine whether session is interactive or batch to facilitate unittest.main(..., exit=testExit)
    import __main__ as main
    testExit = hasattr(main, '__file__')

    sirfReader = SirfReader('test.gpx')

    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

....
----------------------------------------------------------------------
Ran 4 tests in 0.002s

OK
